# 訓練模型

In [ ]:
import numpy as np
import json
from sklearn.model_selection import train_test_split

data = np.load('/content/drive/MyDrive/Colab Notebooks/pluse/alldatas.npy')

# 分割數據為訓練集和驗證集
X_train, X_val = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model, Input, callbacks

# 初始化History回調
history = callbacks.History()

# 編碼器
input_shape = Input(shape=(5000,))  # 輸入為一維長度5000的ndarray
encoder_layer1 = layers.Dense(2048, activation='relu')(input_shape)
encoder_layer2 = layers.Dense(1024, activation='relu')(encoder_layer1)
encoder_layer3 = layers.Dense(512, activation='relu')(encoder_layer2)
encoded_output = layers.Dense(256, activation='relu')(encoder_layer3)

# 解碼器
decoder_layer1 = layers.Dense(512, activation='relu')(encoded_output)
decoder_layer2 = layers.Dense(1024, activation='relu')(decoder_layer1)
decoder_layer3 = layers.Dense(2048, activation='relu')(decoder_layer2)
decoded_output = layers.Dense(5000, activation='tanh')(decoder_layer3)  # 確保輸出層與輸入層相匹配

# 建構自动編碼器模型
autoencoder = Model(inputs=input_shape, outputs=decoded_output)

# 編譯模型
autoencoder.compile(optimizer='adam', loss='mse')

# 初始化EarlyStopping回調
early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=100, verbose=1, mode='min')

# 假設X_train和X_val是預處理好的數據
autoencoder.fit(X_train, X_train, batch_size=512, epochs=1000, callbacks=[early_stopping, history], shuffle=True, validation_data=(X_val, X_val))

# 保存模型
autoencoder.save('models/autoencoder_model', save_format='tf')  # 使用TensorFlow SavedModel格式保存

Epoch 1/1000
4/4 [==============================] - 5s 139ms/step - loss: 0.2486 - val_loss: 0.3374
Epoch 2/1000
4/4 [==============================] - 0s 48ms/step - loss: 0.3029 - val_loss: 0.0825
Epoch 3/1000
4/4 [==============================] - 0s 45ms/step - loss: 0.0666 - val_loss: 0.0499
Epoch 4/1000
4/4 [==============================] - 0s 39ms/step - loss: 0.0458 - val_loss: 0.0438
Epoch 5/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0429 - val_loss: 0.0412
Epoch 6/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0408 - val_loss: 0.0407
Epoch 7/1000
4/4 [==============================] - 0s 38ms/step - loss: 0.0400 - val_loss: 0.0414
Epoch 8/1000
4/4 [==============================] - 0s 46ms/step - loss: 0.0400 - val_loss: 0.0402
Epoch 9/1000
4/4 [==============================] - 0s 41ms/step - loss: 0.0394 - val_loss: 0.0398
Epoch 10/1000
4/4 [==============================] - 0s 44ms/step - loss: 0.0391 - val_loss: 0.0394
Epoch 11

# 將訓練好的模型打包

In [ ]:
import shutil

# 指定要打包的模型資料夾路徑
model_directory = '/content/models'

# 指定壓縮檔案的路徑和名稱
archive_name = '/content/drive/MyDrive/Colab Notebooks/AEmodel/autoencoder_model.zip'

# 使用shutil.make_archive來打包並壓縮資料夾
shutil.make_archive(base_name=archive_name[:-4], format='zip', root_dir=model_directory)


'/content/drive/MyDrive/Colab Notebooks/AEmodel/autoencoder_model.zip'